In [2]:
% pip install git+https://github.com/CAHLR/pyBKT.git

  Cloning https://github.com/CAHLR/pyBKT.git to /tmp/pip-req-build-1zy2ikbu
  Running command git clone -q https://github.com/CAHLR/pyBKT.git /tmp/pip-req-build-1zy2ikbu
  Created wheel for pyBKT: filename=pyBKT-1.0-cp36-cp36m-linux_x86_64.whl size=1260527 sha256=b3148b1bdd5fb03b9d96f3eb8cc1014de696d43939b17cad9546f1ea466ea421
  Stored in directory: /tmp/pip-ephem-wheel-cache-0sd6pr31/wheels/db/9c/7f/7f182d8cd2ba25e21f955dc264e544238eb50dcc41287ceffe
Successfully built pyBKT


In [0]:
#!python test/assistments_train.py #TODO: modify pre-procesd

In [0]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../')
from pyBKT.generate import synthetic_data, random_model_uni
from pyBKT.fit import EM_fit
from pyBKT.util.data_helper import assistments_data
from copy import deepcopy

In [5]:
from google.colab import files
uploaded = files.upload()

Saving ARRS_data.csv to ARRS_data.csv


In [6]:
df = pd.read_csv("ARRS_data.csv")
df = df.rename(columns={"ARRS-check(oneMeansFirstWeek-TwomeansOneweekLater-threemeanstwoweeksafterthat)": "ARRS-check"})
df

,problemlog_id,assignment_id,sequence_id,user_id,ARRS,assistment_id,problem_id,sequence_opp,assignment_opp,ARRS-check,correct,hint_count,attempt_count,first_response_time,overlap_time,cluster,pretest,posttest,avg_initial_three_correctness,avg_initial_#problems_to_master,avg_first_reassess
0,40790855,297282,5959,75063,0,34310,53621,1,1,1,0,0,1,13260,13353,0,0,1,0.671795,6.323529,0.684211
1,40790864,297282,5959,75063,0,34289,53537,2,2,1,1,0,1,4820,4820,0,0,1,0.671795,6.323529,0.684211
2,40790866,297282,5959,75063,0,34295,53561,3,3,1,1,0,1,8340,8340,0,0,1,0.671795,6.323529,0.684211
3,40790867,297282,5959,75063,0,34338,53733,4,4,1,1,0,1,10758,10758,0,0,1,0.671795,6.323529,0.684211
4,40897310,299510,5959,75063,0,34329,53697,5,1,2,1,0,0,10,10,1,0,1,0.671795,6.323529,0.684211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25400,41038106,301286,15675,93241,1,121297,231946,1,1,1,1,0,1,44759,44759,0,1,0,0.630769,5.470588,0.666667
25401,41038288,301286,15675,93241,1,121263,231911,2,2,1,0,0,5,55244,70970,0,1,0,0.630769,5.470588,0.666667
25402,41038533,301286,15675,93241,1,121220,231868,3,3,1,1,0,1,20336,20336,0,1,0,0.630769,5.470588,0.666667
25403,41038618,301286,15675,93241,1,121261,231909,4,4,1,1,0,1,12483,12483,0,1,0,0.630769,5.470588,0.666667


In [7]:
#sort df by problemlog_id to keep the data sorted in a chronological order

#students = df.groupby(["user_id", "sequence_id"]).mean()
df.sort_values("problemlog_id")
df["problem_id"]
len(df["user_id"].unique())

99

In [10]:
#running BKT on Correct column - not sure what the data in that column means though
skills = df["sequence_id"].unique()
for skill in skills:
  print("\n Skill: ", skill)
  num_resources = 0

  num_fit_initializations = 5

  skill_df = df[(df["sequence_id"]==skill)]
  skill_df.loc[:, "correct"] += 1
  df3=skill_df[skill_df["correct"]!=3]
  data=df3["correct"].values

  steps=df3.groupby("user_id")["problem_id"].count().values
  lengths=np.copy(steps)
  steps[0]=0
  steps[1]=1
  for i in range(2,steps.size):
      steps[i]=steps[i-1]+lengths[i-2]

  starts=np.delete(steps,0)

  resources=[1]*data.size
  resource=np.asarray(resources)
  stateseqs=np.copy(resource)
  lengths=np.resize(lengths,lengths.size-1)
  Data={}
  Data["stateseqs"]=np.asarray([stateseqs],dtype='int32')
  Data["data"]=np.asarray([data],dtype='int32')
  Data["starts"]=np.asarray(starts)
  Data["lengths"]=np.asarray(lengths)
  Data["resources"]=resource

  num_gs = 1 #number of guess/slip classes
  num_learns = 1 #number of learning rates

  print('fitting! each dot is a new EM initialization')

  best_likelihood = float("-inf")

  for i in range(num_fit_initializations):
    fitmodel = random_model_uni.random_model_uni(num_learns, num_gs) # include this line to randomly set initial param values
    (fitmodel, log_likelihoods) = EM_fit.EM_fit(fitmodel, Data)
    if(log_likelihoods[-1] > best_likelihood):
      best_likelihood = log_likelihoods[-1]
      best_model = fitmodel

  print('')
  print('\tlearned')
  print('prior\t%.4f' % (best_model["pi_0"][1][0]))
  for r in range(num_learns):
      print('learn%d\t%.4f' % (r+1, best_model['As'][r, 1, 0].squeeze()))
  for r in range(num_learns):
      print('forget%d\t%.4f' % (r+1, best_model['As'][r, 0, 1].squeeze()))

  for s in range(num_gs):
      print('guess%d\t%.4f' % (s+1, best_model['guesses'][s]))
  for s in range(num_gs):
      print('slip%d\t%.4f' % (s+1, best_model['slips'][s]))


 Skill:  5959
fitting! each dot is a new EM initialization

	learned
prior	0.9154
learn1	0.6617
forget1	0.0000
guess1	0.0014
slip1	0.0613

 Skill:  5962
fitting! each dot is a new EM initialization

	learned
prior	0.7040
learn1	0.2342
forget1	0.0000
guess1	0.6390
slip1	0.0441

 Skill:  6018
fitting! each dot is a new EM initialization


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s



	learned
prior	0.9533
learn1	0.0871
forget1	0.0000
guess1	0.2657
slip1	0.1619

 Skill:  6057
fitting! each dot is a new EM initialization

	learned
prior	0.6795
learn1	0.6112
forget1	0.0000
guess1	0.5810
slip1	0.1085

 Skill:  6848
fitting! each dot is a new EM initialization

	learned
prior	0.7904
learn1	0.4642
forget1	0.0000
guess1	0.5358
slip1	0.0663

 Skill:  6851
fitting! each dot is a new EM initialization

	learned
prior	0.6455
learn1	0.4943
forget1	0.0000
guess1	0.0856
slip1	0.1784

 Skill:  6854
fitting! each dot is a new EM initialization

	learned
prior	0.8047
learn1	0.1796
forget1	0.0000
guess1	0.6152
slip1	0.0950

 Skill:  6876
fitting! each dot is a new EM initialization

	learned
prior	0.9516
learn1	0.5694
forget1	0.0000
guess1	0.5915
slip1	0.0500

 Skill:  6895
fitting! each dot is a new EM initialization

	learned
prior	0.6195
learn1	0.8313
forget1	0.0000
guess1	0.5529
slip1	0.1573

 Skill:  6915
fitting! each dot is a new EM initialization

	learned
prior	0.9339
lear